### Importing packages and data

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
import random
import re

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_colwidth', 20)
pd.set_option('display.width', 20)

data = pd.read_csv("C:\\Users\\marci\\Downloads\\real_estate_ads 3.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\marci\\Downloads\\real_estate_ads 3.csv'

Here we defined function that extracts parameters from column `params`. We found that is was coded in a way where we have name of the parameter followed by `<=>` and then value of the parameter. If more than one value was present they were separated by `<->`.

In [ ]:
def process_params(params_str):
    params = {}
    values = params_str.split('<br>')
    for value in values:
        if '<=>' in value:
            param, val = value.split('<=>')
            if '<->' in val:
                val = val.split('<->')
            params[param] = val
    return params
    
df_params = data['params'].apply(process_params).apply(pd.Series)

data_new = pd.concat([data, df_params], axis=1)

data_new = data_new.loc[:, ~data_new.columns.duplicated()]

Some of the parameters in some columns were coded in a problematic way, for example `p,a,r,a,m,e,t,e,r` which is the same as `parameter`. To deal with this problem, we created a function that checks if there are values of parameters that are one letter and then changes them into normal coding, so from `p,a,r,a,m,e,t,e,r` we will go to `parameter`.

In [79]:
target_columns = ['rent[currency]', 'building_ownership', 'media_types', 'security_types',
                  'equipment_types', 'extras_types', 'remote_services', 'market', 'rooms_num', 'building_type',
                  'floor_no', 'building_floors_num', 'building_material', 'windows_type', 'heating', 'rent',
                  'no_rooms', 'construction_status']

unique_values = {}


for col in target_columns:

    if data_new[col].apply(type).eq(list).any():
        data_new[col] = data_new[col].apply(lambda x: ','.join(x))

    unique_values[col] = data_new[col].unique()

data_new = data_new.drop('params', axis=1)

prob_columns = ['extras_types', 'equipment_types', 'media_types', 'security_types']

def process_values(value):
    if ',' in value:
        parts = value.split(',')
        parts = [part.strip() for part in parts]
        if all(len(part) == 1 for part in parts):
            processed_value = ''.join(parts)
        else:
            processed_value = value
    else:
        processed_value = value
    return processed_value



for col in prob_columns:
    data_new[col] = data_new[col].apply(process_values)

We created a function that sorts parameter values in alphabetical order, which we thought might be useful later.

In [80]:
def process_values(value):
    if pd.notna(value) and isinstance(value, str) and ',' in value:
        parts = value.split(',')
        parts = [part.strip() for part in parts]
        parts.sort() 
        processed_value = ','.join(parts)
    else:
        processed_value = value
    return processed_value

for col in prob_columns:
    data_new[col] = data_new[col].apply(process_values)

Because there were three different curriencies of rent, we decided to change everything to PLN. Exchange rates are from Google. Because of this change, we can drop columns `rent[currency]`.

In [81]:
data_new['rent'] = pd.to_numeric(data_new['rent'], errors='coerce')
def convert_currency(row):
    if row['rent[currency]'] == 'USD':
        return row['rent'] * 3.94
    elif row['rent[currency]'] == 'EUR':
        return row['rent'] * 4.29
    else:
        return row['rent']

data_new['rent'] = data_new.apply(convert_currency, axis=1)

data_new = data_new.drop(['rent[currency]', 'no_rooms'],axis=1)

Most of the columns needed some sort of mapping because of their form. Here we did it by hand just to keep track what is coded as which value. We could also do this by using LabelEncoder and then creating a dictionary, but from my expirience this dictionary sometimes is not that easy to read and create, so we created mapping by hand. Of course for bigger problems we would use LabelEncoder.

In [82]:
data_new['market'] = data_new['market'].replace({'primary': 1, 'secondary': 2})
data_new['rooms_num'] = data_new['rooms_num'].replace({'more': 8})
data_new['building_type'] = data_new['building_type'].replace({'block': 1, 'apartment': 2, 'ribbon': 3, 'tenement': 4, 'house': 5})
data_new['floor_no'] = data_new['floor_no'].replace({'ground_floor': 0, 
                                                 'floor_1': 1, 
                                                 'floor_2': 2, 
                                                 'floor_3': 3, 
                                                 'floor_4': 4,
                                                 'floor_5': 5,
                                                 'floor_6': 6,
                                                 'floor_7': 7,
                                                 'floor_8': 8,
                                                 'floor_9': 9,
                                                 'floor_10': 10,
                                                 'floor_higher_10':11,
                                                 'cellar': -1,
                                                 'garret': 20})
data_new['building_floors_num'] = pd.to_numeric(data_new['building_floors_num'], errors='coerce')
data_new['building_floors_num'] = data_new['building_floors_num'].replace(110, 10)
data_new['building_material'] = data_new['building_material'].replace({'brick': 1, 'concrete_plate': 2, 'other': 3, 'breezeblock': 4, 'reinforced_concrete': 5,
                                                                  'silikat':6, 'concrete': 7, 'cellular_concrete': 8, 'hydroton': 9, 'wood': 10})
data_new['windows_type'] = data_new['windows_type'].replace({'plastic': 1, 'wooden': 2, 'aluminium': 3})
data_new['heating'] = data_new['heating'].replace({'urban': 1, 'other': 2, 'gas': 3, 'boiler_room': 4, 'electrical': 5, 'tiled_stove': 6})
data_new['construction_status'] = data_new['construction_status'].replace({'to_completion': 1, 'ready_to_use': 2, 'to_renovation': 3})
data_new['building_ownership'] = data_new['building_ownership'].replace({'co_operative_ownership_with_a_land_and_mortgage_registe': 3,
    'co_operative_ownership': 2,
    'full_ownership': 1,
    'share': 4})

In [83]:
data_new.columns

Index(['id',
       'market',
       'created_at_first',
       'created_at',
       'district_lon',
       'district_lat',
       'title',
       'description',
       'price',
       'm',
       'price_per_m',
       'map_lon',
       'map_lat',
       'price[currency]',
       'rooms_num',
       'building_type',
       'floor_no',
       'building_floors_num',
       'building_material',
       'windows_type',
       'heating',
       'build_year',
       'construction_status',
       'rent',
       'building_ownership',
       'free_from',
       'media_types',
       'security_types',
       'equipment_types',
       'extras_types',
       'remote_services'],
      dtype='object')

Now we wanted to extract information further. We wanted to split parameters with more than one value in such a way we can decide whether given apartment has parameter X or not. For example, in column `extras_types` there are different types of extra equipment such as balcony, lift, garden, etc. We wanted to extract each of these extras and assign them to unique columns in the dataset. Then if the given apartment has a balcony we assign 1 in column balcony and 0 if not. We repeat this for every colum that has multiple values of the parameter paying extra attention to `NaN` values.

In [84]:
data_new[['media_types',
       'security_types',
       'equipment_types',
       'extras_types']].head()

,media_types,security_types,equipment_types,extras_types
0,cable-television...,,,"balcony,separate..."
1,cable-television...,,,balcony
2,,entryphone,,"balcony,basement"
3,,entryphone,,"balcony,basement"
4,cable-television...,entryphone,"fridge,furniture...",basement


In [85]:
data_new['extras_types'].value_counts()

extras_types
balcony,lift                                 1951
balcony,garage,lift                          1594
                                             1111
balcony,basement,separate_kitchen             629
balcony,basement,lift                         613
                                             ... 
balcony,two_storey                              1
basement,garden,lift,terrace                    1
air_conditioning,balcony,separate_kitchen       1
garden,lift,terrace,usable_room                 1
air_conditioning                                1
Name: count, Length: 265, dtype: int64

In [86]:
unique_values = data_new['extras_types'].str.split(',').explode().unique()

all_unique_elements = list(unique_values)

extras_types_dict = {element: [] for element in all_unique_elements}

print(extras_types_dict)

{'balcony': [], 'separate_kitchen': [], 'basement': [], 'lift': [], '': [], 'garage': [], 'garden': [], 'air_conditioning': [], 'terrace': [], 'usable_room': [], '0': [], 'two_storey': []}


In [87]:
df_extras = pd.DataFrame()

for col in extras_types_dict:
    df_extras[col] = data_new['extras_types'].str.contains(col, na=False).astype(int)

print(df_extras.columns)
df_extras.head()

Index(['balcony',
       'separate_kitchen',
       'basement',
       'lift', '',
       'garage',
       'garden',
       'air_conditioning',
       'terrace',
       'usable_room',
       '0',
       'two_storey'],
      dtype='object')


,balcony,separate_kitchen,basement,lift,,garage,garden,air_conditioning,terrace,usable_room,0,two_storey
0,1,1,0,0,1,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,0,0
2,1,0,1,0,1,0,0,0,0,0,0,0
3,1,0,1,0,1,0,0,0,0,0,0,0
4,0,0,1,0,1,0,0,0,0,0,0,0


In [88]:
df_extras.drop(df_extras.columns[4],axis=1, inplace=True)
df_extras.drop(df_extras.columns[9],axis=1,inplace=True)
df_extras.columns

Index(['balcony',
       'separate_kitchen',
       'basement',
       'lift',
       'garage',
       'garden',
       'air_conditioning',
       'terrace',
       'usable_room',
       'two_storey'],
      dtype='object')

In [89]:
data_new.drop('extras_types', axis=1, inplace=True)

data_new = pd.concat([data_new, df_extras], axis=1)

In [90]:
data_new.columns

Index(['id',
       'market',
       'created_at_first',
       'created_at',
       'district_lon',
       'district_lat',
       'title',
       'description',
       'price',
       'm',
       'price_per_m',
       'map_lon',
       'map_lat',
       'price[currency]',
       'rooms_num',
       'building_type',
       'floor_no',
       'building_floors_num',
       'building_material',
       'windows_type',
       'heating',
       'build_year',
       'construction_status',
       'rent',
       'building_ownership',
       'free_from',
       'media_types',
       'security_types',
       'equipment_types',
       'remote_services',
       'balcony',
       'separate_kitchen',
       'basement',
       'lift',
       'garage',
       'garden',
       'air_conditioning',
       'terrace',
       'usable_room',
       'two_storey'],
      dtype='object')

In [91]:
unique_values = data_new['media_types'].str.split(',').explode().unique()

all_unique_elements = list(unique_values)

media_types_dict = {element: [] for element in all_unique_elements}

print(media_types_dict)

df_media = pd.DataFrame()

for col in media_types_dict:
    df_media[col] = data_new['media_types'].str.contains(col, na=False).astype(int)

df_media.drop(df_media.columns[3],axis=1, inplace=True)
df_media.drop(df_media.columns[3],axis=1, inplace=True)

data_new.drop('media_types', axis=1, inplace=True)

data_new = pd.concat([data_new, df_media], axis=1)
print(df_media.columns)
data_new.columns

{'cable-television': [], 'internet': [], 'phone': [], '': [], '0': []}
Index(['cable-television',
       'internet',
       'phone'],
      dtype='object')


Index(['id',
       'market',
       'created_at_first',
       'created_at',
       'district_lon',
       'district_lat',
       'title',
       'description',
       'price',
       'm',
       'price_per_m',
       'map_lon',
       'map_lat',
       'price[currency]',
       'rooms_num',
       'building_type',
       'floor_no',
       'building_floors_num',
       'building_material',
       'windows_type',
       'heating',
       'build_year',
       'construction_status',
       'rent',
       'building_ownership',
       'free_from',
       'security_types',
       'equipment_types',
       'remote_services',
       'balcony',
       'separate_kitchen',
       'basement',
       'lift',
       'garage',
       'garden',
       'air_conditioning',
       'terrace',
       'usable_room',
       'two_storey',
       'cable-television',
       'internet',
       'phone'],
      dtype='object')

In [92]:
unique_values = data_new['equipment_types'].str.split(',').explode().unique()

all_unique_elements = list(unique_values)

eq_types_dict = {element: [] for element in all_unique_elements}

print(eq_types_dict)

df_eq = pd.DataFrame()

for col in eq_types_dict:
    df_eq[col] = data_new['equipment_types'].str.contains(col, na=False).astype(int)

df_eq.drop(df_eq.columns[8],axis=1, inplace=True)
df_eq.drop(df_eq.columns[0],axis=1, inplace=True)

data_new.drop('equipment_types', axis=1, inplace=True)

data_new = pd.concat([data_new, df_eq], axis=1)
print(df_eq.columns)
data_new.columns

{'': [], 'fridge': [], 'furniture': [], 'stove': [], 'washing_machine': [], 'oven': [], 'dishwasher': [], 'tv': [], '0': []}
Index(['fridge',
       'furniture',
       'stove',
       'washing_machine',
       'oven',
       'dishwasher',
       'tv'],
      dtype='object')


Index(['id',
       'market',
       'created_at_first',
       'created_at',
       'district_lon',
       'district_lat',
       'title',
       'description',
       'price',
       'm',
       'price_per_m',
       'map_lon',
       'map_lat',
       'price[currency]',
       'rooms_num',
       'building_type',
       'floor_no',
       'building_floors_num',
       'building_material',
       'windows_type',
       'heating',
       'build_year',
       'construction_status',
       'rent',
       'building_ownership',
       'free_from',
       'security_types',
       'remote_services',
       'balcony',
       'separate_kitchen',
       'basement',
       'lift',
       'garage',
       'garden',
       'air_conditioning',
       'terrace',
       'usable_room',
       'two_storey',
       'cable-television',
       'internet',
       'phone',
       'fridge',
       'furniture',
       'stove',
       'washing_machine',
       'oven',
       'dishwasher',
       'tv'],
      

In [93]:
unique_values = data_new['security_types'].str.split(',').explode().unique()

all_unique_elements = list(unique_values)

sec_types_dict = {element: [] for element in all_unique_elements}

print(sec_types_dict)

df_sec = pd.DataFrame()

for col in sec_types_dict:
    df_sec[col] = data_new['security_types'].str.contains(col, na=False).astype(int)

df_sec.drop(df_sec.columns[2],axis=1, inplace=True)
df_sec.drop(df_sec.columns[0],axis=1, inplace=True)

data_new.drop('security_types', axis=1, inplace=True)

data_new = pd.concat([data_new, df_sec], axis=1)
print(df_sec.columns)
data_new.columns

{'': [], 'entryphone': [], '0': [], 'closed_area': [], 'monitoring': [], 'anti_burglary_door': [], 'roller_shutters': [], 'alarm': []}
Index(['entryphone',
       'closed_area',
       'monitoring',
       'anti_burglary_door',
       'roller_shutters',
       'alarm'],
      dtype='object')


Index(['id',
       'market',
       'created_at_first',
       'created_at',
       'district_lon',
       'district_lat',
       'title',
       'description',
       'price',
       'm',
       'price_per_m',
       'map_lon',
       'map_lat',
       'price[currency]',
       'rooms_num',
       'building_type',
       'floor_no',
       'building_floors_num',
       'building_material',
       'windows_type',
       'heating',
       'build_year',
       'construction_status',
       'rent',
       'building_ownership',
       'free_from',
       'remote_services',
       'balcony',
       'separate_kitchen',
       'basement',
       'lift',
       'garage',
       'garden',
       'air_conditioning',
       'terrace',
       'usable_room',
       'two_storey',
       'cable-television',
       'internet',
       'phone',
       'fridge',
       'furniture',
       'stove',
       'washing_machine',
       'oven',
       'dishwasher',
       'tv',
       'entryphone',
       'clo

In [94]:
data_new.head()

,id,market,created_at_first,created_at,district_lon,district_lat,title,description,price,m,...,washing_machine,oven,dishwasher,tv,entryphone,closed_area,monitoring,anti_burglary_door,roller_shutters,alarm
0,49341,2,2018-01-04 13:11:05,2018-01-08 11:27:35,16.9408,52.38356,NaN,<p>Zapraszam do ...,289000.0,48.000000,...,0,0,0,0,0,0,0,0,0,0
1,261720,2,2018-01-15 19:12:07,2018-01-25 11:11:31,16.9408,52.38356,NaN,<p>Zapraszam do ...,379000.0,61.000000,...,0,0,0,0,0,0,0,0,0,0
2,50441,2,2018-01-04 13:53:29,2018-01-11 05:55:12,16.9408,52.38356,Rataje Nowe Mias...,<p>Weychan Nieru...,339000.0,63.000000,...,0,0,0,0,1,0,0,0,0,0
3,203518,2,2018-01-12 11:10:57,2018-01-30 14:38:39,16.9408,52.38356,Mieszkanie 3 pok...,<p>Oferuję na sp...,330000.0,59.200001,...,0,0,0,0,1,0,0,0,0,0
4,5,2,2018-01-02 11:34:51,2018-01-18 14:22:29,16.9408,52.38356,Inwestycja -rond...,<p><strong>Dwa p...,259000.0,38.000000,...,1,0,0,0,1,0,0,0,0,0


In [99]:
data_new.isna().sum().sort_values(ascending=False)

rent                   9898
remote_services        9520
building_floors_num    2088
title                   927
district_lon            187
                       ... 
construction_status       0
building_ownership        0
free_from                 0
market                    0
alarm                     0
Length: 53, dtype: int64

Finally, the `params` column is broken into additional columns that can help us create a model.

In [100]:
# data_new.to_csv('extracted_data.csv', index=False)